In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# CSAF Imports
import csaf.config as cconf
import csaf.system as csys

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_multiagent_central_config.toml")

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

In [ ]:
def air_collision_condition(ctraces):
        """ground collision premature termnation condition
        TODO: I had to alter the terminating conditions to support this expression
        """
        # get the aircraft states
        sa, sb = ctraces['planta']['states'], ctraces['plantb']['states']
        if sa and sb:
            # look at distance between last state
            return (np.linalg.norm(np.array(sa[-1][9:11]) - np.array(sb[-1][9:11]))) < 10

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

# create an environment from the system, allowing us to act as the controller
my_env = csys.SystemEnv("autopilot", my_system, terminating_conditions_all=air_collision_condition)

In [ ]:
# set the scenario states
my_system.set_state('planta', [500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6000.0, 9.0])
my_system.set_state('plantb', [500.0, 0.0, 0.0, 0.0, 0.0, np.pi, 0.0, 0.0, 0.0, 2000.0, 0.0, 6000.0, 9.0])

In [ ]:
# collect aircraft states
pstates = []

# send signal of zeros
ctrl_signal = [0.,0.,0.,0.]

# step through simulation and collect f16 states
# StopIteration is thrown when the terminating conditions are achieved
do_sim = True
while do_sim:
    try:
        comp_input = my_env.step({"autopilot-outputsa": ctrl_signal, "autopilot-outputsb": ctrl_signal}) 
        pstates.append((comp_input['planta-states'], comp_input['plantb-states']))
        print(pstates[-1][0])
    except StopIteration as e:
        print(e)
        do_sim = False
        break
    except Exception as e:
        raise e
        break
pstates = np.array(pstates)